## Import libraries

In [1]:
import sys
import numpy as np
import pandas as pd
import hypertools as hyp
from os.path import abspath, join as opj
from scipy.stats import pearsonr, sem
from scipy.interpolate import interp1d

%matplotlib inline

## Import analysis helpers

In [2]:
sys.path.insert(0, abspath('../../helpers/'))
from analysis_helpers import (
    N_TOPICS,
    VIDEO_WSIZE,
    RECALL_WSIZE,
    VECTORIZER_PARAMS,
    SEMANTIC_PARAMS,
    format_text,
    parse_windows,
    get_video_timepoints,
    warp_recall,
    create_diag_mask
)

Functions and variables used across multiple notebooks can be found [here](https://github.com/contextlab/sherlock-topic-model-paper/blob/master/code/helpers/analysis_helpers.py)

## Set paths & parameters

In [3]:
rawdir = '../../../data/raw/'
datadir = '../../../data/processed/'

## Define some functions

In [4]:
# wrap full topic modeling pipeline
def transform_video(annotations):
    dropcols = ['Start Time (s) ', 'End Time (s) ', 
                'Start Time (TRs, 1.5s)', 'End Time (TRs, 1.5s)']
    features = annotations.drop(columns=dropcols)
    scenes_list = features.apply(format_text, axis=1).tolist()
    video_windows, window_bounds = parse_windows(scenes_list, VIDEO_WSIZE)
    
    video_model = hyp.tools.format_data(video_windows, 
                                    vectorizer=VECTORIZER_PARAMS, 
                                    semantic=SEMANTIC_PARAMS, 
                                    corpus=video_windows)[0]
    
    tr_spans = video_text[['Start Time (TRs, 1.5s)', 'End Time (TRs, 1.5s)']]
    starts, stops = tr_spans.values.T
    video_model_TRs = np.empty((1976, 100))
    xvals = get_video_timepoints(window_bounds, annotations)
    xvals_TR = np.array(xvals) * 1976 / 2963
    TR_times = np.arange(1, 1977)
    interp_func = interp1d(xvals_TR, video_model, axis=0, fill_value='extrapolate')
    video_model_TRs = interp_func(TR_times)
    return video_model_TRs, video_windows

In [5]:
def transform_recalls(recall_windows, video_windows, video_traj):
    recall_models = hyp.tools.format_data(recall_windows, 
                                          vectorizer=VECTORIZER_PARAMS, 
                                          semantic=SEMANTIC_PARAMS, 
                                          corpus=video_windows)
    # warp recall trajectores to video trajectory length
    recalls_warped = [warp_recall(r, video_traj, return_paths=False) 
                      for r in recall_models]
    return recalls_warped

In [10]:
def correlate_structures(video, other):
    assert video.shape == other.shape
    vcorr = np.corrcoef(video)
    ocorr = np.corrcoef(other)
    diag_mask = create_diag_mask(vcorr, diag_limit=238)  # precomputed from intact video
    v = vcorr[diag_mask]
    o = ocorr[diag_mask]
    return pearsonr(v, o)[0]

## Load & format data

In [7]:
video_text = pd.read_excel(opj(rawdir, 'Sherlock_Segments_1000_NN_2017.xlsx'))
video_text['Scene Segments'].fillna(method='ffill', inplace=True)
video_text.drop(index=[480, 481], inplace=True)
video_text.reset_index(drop=True, inplace=True)
video_text.loc[480:, 'Start Time (s) ': 'End Time (s) '] += video_text.loc[479, 'End Time (s) ']
keep_cols = np.append(video_text.columns[1:5], video_text.columns[6:15])
video_text = video_text.loc[:, keep_cols]
video_text.columns = list(video_text.columns[:4]) + ['Narrative details', 'Indoor vs outdoor', 
                                                     'Characters on screen', 'Character in focus', 
                                                     'Character speaking', 'Location', 'Camera angle', 
                                                     'Music presence', 'Text on screen']

# trajectories created from all features
full_trajs = np.load(opj(datadir, 'models_t100_v50_r10.npy'), allow_pickle=True)
full_video, full_recalls = full_trajs

In [8]:
recall_w = []
for sub in range(1, 18):
    transcript_path = opj(rawdir, f'NN{sub} transcript.txt')
    with open(transcript_path, 'r', encoding='cp1252') as f:
        recall = f.read().replace(b'\x92'.decode('cp1252'), "'").strip()
    recall_fmt = format_text(recall).split('.')
    if not recall_fmt[-1]:
        recall_fmt = recall_fmt[:-1]
    sub_recall_w = parse_windows(recall_fmt, RECALL_WSIZE)[0]
    recall_w.append(sub_recall_w)

## Iteratively hold out one feature and transform remaining

In [27]:
features = video_text.columns[4:]
# dropfeat_corrs = dict.fromkeys(features)
analyses = ['full vid corr', 'vid rec corr', 'vid rec sem']
dropfeat_corrs = pd.DataFrame(index=features, columns=analyses)

for feature in features:
    print(f'{feature}:')
    # transform remaining annotations
    other_features = video_text.drop(feature, axis=1)
#     other_features = partial_df.loc[:, partial_df.columns[4:]]
    dropfeat_vid, dropfeat_vid_ws = transform_video(other_features)
    
    # compute similarity with full-feature video trajectory structure
    full_video_corr = correlate_structures(full_video, dropfeat_vid)
    
    # transform recalls using feature-removed corpus
    dropfeat_recs = transform_recalls(recall_w, dropfeat_vid_ws, dropfeat_vid)
    
    # compare structures to partial video model
    rec_corrs = np.array([correlate_structures(dropfeat_vid, r) 
                          for r in dropfeat_recs])
    feat_corr, feat_sem = rec_corrs.mean(), sem(rec_corrs)

    dropfeat_corrs.loc[feature] = [full_video_corr, feat_corr, feat_sem]
    print(f'\tsimilarity to full video: {full_video_corr}')
    print(f'\tvideo-recall structure similarity: {feat_corr}, SEM: {feat_sem}\n')
    
# add data for full model
rec_corr_full = np.array([correlate_structures(full_video, warp_recall(r, full_video, return_paths=False)) 
                          for r in full_recalls])
dropfeat_corrs.loc['All features'] = [1, rec_corr_full.mean(), sem(rec_corr_full)]
print('All features')
print(f'\tvideo-recall structure similarity: {rec_corr_full.mean()}, SEM: {sem(rec_corr_full)}')

Narrative details:
	similarity to full video: 0.7840464784236999
	video-recall structure similarity: 0.49789433751038686, SEM: 0.03575362265021856

Indoor vs outdoor:
	similarity to full video: 0.8576974275883418
	video-recall structure similarity: 0.6569000158177016, SEM: 0.027134409621705116

Characters on screen:
	similarity to full video: 0.7976882761325998
	video-recall structure similarity: 0.6818517096348622, SEM: 0.027818639564926455

Character in focus:
	similarity to full video: 0.873694587735584
	video-recall structure similarity: 0.5982538568209149, SEM: 0.03024753485130499

Character speaking:
	similarity to full video: 0.8651720030104021
	video-recall structure similarity: 0.6303607927331502, SEM: 0.03204785143227152

Location:
	similarity to full video: 0.7871552037453584
	video-recall structure similarity: 0.68326123617632, SEM: 0.025844925341076984

Camera angle:
	similarity to full video: 0.8431489481128159
	video-recall structure similarity: 0.6603313082637992, SEM: 

In [28]:
# dropfeat_corrs.to_pickle(opj(datadir, 'feature_contribution.p'))